In [6]:
import pandas as pd

gs = pd.read_feather("/home/lisa/data/llm_ensembles/results/ft/experiments/llama3_7ms_6ch2500.arrow")
print(gs.columns)
preds = pd.read_feather("/home/lisa/data/llm_ensembles/results/ft/experiments/llama3_7ms.arrow")
preds = preds[["doc_id", "label_id"]]
preds.columns = ["doc_id", "label_id"]
print(preds.columns)
print(gs.shape, preds.shape)

Index(['doc_id', 'label_id', 'score'], dtype='object')
Index(['doc_id', 'label_id'], dtype='object')
(214582, 3) (42055, 2)


In [7]:
merged_df = pd.merge(preds, gs, on='doc_id', suffixes=('_pred', '_gs'))

In [8]:
merged_df.shape

(1081119, 4)

In [11]:
merged_df.head(25)

,doc_id,label_id_pred,label_id_gs,score
0,1122545479,960767053,041220447,1
1,1122545479,960767053,041429036,1
2,1122545479,960767053,041429036,1
3,1122545479,960767053,964751038,1
4,1122545479,960767053,040039579,1
5,1122545479,960767053,040039579,2
6,1122545479,960767053,041242610,1
7,1122545479,960767053,042102383,1
8,1122545479,960767053,941101223,1
9,1122545479,960767053,040124940,1


In [1]:
from prompt_template import PromptBuilder

pf = "/home/lisa/repos/semeval/assets/prompts/abstracts/abstracts_prompt_7ms.txt"
ci = "/home/lisa/repos/semeval/assets/prompts/instructions/instruction01.txt"
llm = "meta-llama/Meta-Llama-3.1-8B-Instruct"
pb_dir = "/home/lisa/repos/semeval/assets/prompts/templates/"
tf = "/home/lisa/repos/semeval/assets/prompts/templates/llama.json"

pb = PromptBuilder(pf, llm, ci)
prompt = pb.build_prompt_by_template(tf, debug=True)



<|begin_of_text|><|start_header_id|>system<|end_header_id|>

/home/lisa/repos/semeval/assets/prompts/instructions/instruction01.txt
<|eot_id|><|start_header_id|>user<|end_header_id|>

Extrahiere Schlagwörter aus Texten.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

nan<|eot_id|><|start_header_id|>user<|end_header_id|>

Text: Die vorliegende Arbeit behandelt Anwendungen von Lenkradvibrationen im Kraftfahrzeug, wobei zur Abgrenzung vom Stand der Technikexplizit taktile, über Sensoren der Haut wahrnehmbare Anregungen fokussiert werden. Derartigen haptischen Signalen in der Lenkung wird aufgrund der Kompatibilität von Reiz- und Handlungsort ein Potential zur Beeinflussung von Fahrerreaktionen bezüglich der Fahrzeugquerführung beigemessen. Als besondere Eigenschaft vibrotaktiler Anregungen wird die Möglichkeit zur Erzeugung örtlich aufgelöster Vibrationen erkannt, die Kommunikation von Richtungsinformation an den Fahrer ermöglicht. Aus dem Stand der Technik wird die subjek

In [2]:
from prompt_template import PromptBuilder

pf = "/home/lisa/repos/semeval/assets/prompts/abstracts/abstracts_prompt_7ms.txt"
cif = "/home/lisa/repos/semeval/assets/prompts/instructions/instruction01.txt"
with open(cif) as f:
    ci = f.read()

llm = "meta-llama/Meta-Llama-3.1-8B-Instruct"
pb_dir = "/home/lisa/repos/semeval/assets/prompts/templates/"
tf = "/home/lisa/repos/semeval/assets/prompts/templates/llama.json"

pb = PromptBuilder(pf, llm, ci)
prompt = pb.build_prompt_by_template(tf, debug=True)

Instruction: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

{custom_instruction}
<|eot_id|><|start_header_id|>user<|end_header_id|>


Example_pattern: {text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Keywords pattern: {keywords}<|eot_id|><|start_header_id|>user<|end_header_id|>


Test item pattern: {text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Prompt after instruction:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent antwortet mit Schlagwörtern auf den Text des Nutzers.
<|eot_id|><|start_header_id|>user<|end_header_id|>


Prompt after example:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent antwortet mit Schlagwörtern auf den Text des Nutzers.
<|eot_id|><|start_header_id|>user<|end_

# Fine-Tuning (from deeplearning.ai)

In [ ]:
# data prep

import pandas as pd
import datasets

from pprint import pprint
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")

def tokenize_function(examples):
    if "question" in examples and "answer" in examples:
      text = examples["question"][0] + examples["answer"][0]
    elif "input" in examples and "output" in examples:
      text = examples["input"][0] + examples["output"][0]
    else:
      text = examples["text"][0]

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=max_length
    )

    return tokenized_inputs


finetuning_dataset_loaded = datasets.load_dataset("json", data_files=filename, split="train")

tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)

print(tokenized_dataset)

tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
print(split_dataset)

# This is how to push your own dataset to your Huggingface hub
# !pip install huggingface_hub
# !huggingface-cli login
# split_dataset.push_to_hub(dataset_path_hf)

In [ ]:
# training

#lamini
import os
import lamini

lamini.api_url = os.getenv("POWERML__PRODUCTION__URL")
lamini.api_key = os.getenv("POWERML__PRODUCTION__KEY")
from llama import BasicModelRunner

model = BasicModelRunner("EleutherAI/pythia-410m") 
model.load_data_from_jsonlines("lamini_docs.jsonl", input_key="question", output_key="answer")
model.train(is_public=True) 

## regular

import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner


logger = logging.getLogger(__name__)
global_config = None

dataset_path = "lamini/lamini_docs"
use_hf = True
model_name = "EleutherAI/pythia-70m"

training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

base_model = AutoModelForCausalLM.from_pretrained(model_name)
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")
base_model.to(device)

def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

# TRAIN
max_steps = 3
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

training_output = trainer.train()

save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)
finetuned_slightly_model.to(device) 




# Test playground

In [2]:
import requests
import json

class Completion: 
    def __init__(self, hyperparameters: dict, llm: str):
        self.debug=False
        self.hyperparameters = hyperparameters
        self.model = llm
        self.complete_url = "http://localhost:" + '9231' + "/v1/completions"
        # print("Complete URL:", self.complete_url)    

    def completion(self, prompt: str):
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
            #  "Authorization": self.authorization, -- not needed ?
        }

        payload = json.dumps(
            {
                "model": self.model,
                "prompt": prompt,
                "max_tokens": self.hyperparameters.get("max_new_tokens", 64),
                "temperature": self.hyperparameters.get("temperature", 0),
            }
        )

        response = requests.post(self.complete_url, headers=headers, data=payload)
        # print(response.json())
        i=0
        success = False
        while i < 10:
            try:
                # response = response.json()
                answer_text = response.json()["choices"][0]["text"]
                answer = answer_text.split("\n")[0].split("<")[0]
                # print("ANSWER: ", answer)
                result = [x.strip() for x in answer.split(", ")]
                if self.debug:
                    print("resuult:",result)
                success = True
                break
            except:
                response = requests.post(self.complete_url, headers=headers, data=payload)
                i = i+1
        pruned_results = []
        if success == False:
            result = ["ERROR404"]
            print("Error!")
        else:
            for res_item in result:
                if len(res_item )<2:
                    # logging.warning("Empty result")
                    pass
                else:
                    pruned_results.append(res_item)
            if len(pruned_results) == 0:
                logging.warning("Results entirely empty")
                result = ["ERROR404"]
        assert all([len(x)>0 for x in pruned_results]), "Some of the results are empty"
        # print("pruned results", pruned_results)
        if self.debug:
            # if success==False:
            print("Prompt: ", prompt)
            print("Success: ", success)
            print("Response: ", response.content)
            print("Result: ", result)
            print("Pruned results: ", pruned_results)
            print("-------------")
        return list(set(pruned_results))

In [36]:
import requests
import json

class Completion_Multiple: 
    def __init__(self, hyperparameters: dict, llm: str):
        self.debug=False
        self.hyperparameters = hyperparameters
        self.model = llm
        self.complete_url = "http://localhost:" + '9231' + "/v1/completions"
        # print("Complete URL:", self.complete_url)    

    def completion(self, prompt: str):
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
            #  "Authorization": self.authorization, -- not needed ?
        }

        payload = json.dumps(
            {
                "model": self.model,
                "prompt": prompt,
                "max_tokens": self.hyperparameters.get("max_new_tokens", 64),
                "temperature": self.hyperparameters.get("temperature", 0),
                "best_of": self.hyperparameters.get("best_of", 1)
            }
        )

        response = requests.post(self.complete_url, headers=headers, data=payload)
        # print(response.json())
        i=0
        success = False
        while i < 10:
            try:
                # response = response.json()
                # print("response.json:", response.json())
                answer_text = [x["text"] for x in response.json()["choices"]]
                print("AT", answer_text)
                answer = answer_text.split("\n")[0].split("<")[0]
                # print("ANSWER: ", answer)
                result = [x.strip() for x in answer.split(", ")]
                if self.debug:
                    print("resuult:",result)
                success = True
                break
            except:
                response = requests.post(self.complete_url, headers=headers, data=payload)
                i = i+1
        pruned_results = []
        if success == False:
            result = ["ERROR404"]
            print("Error!")
        else:
            for res_item in result:
                if len(res_item )<2:
                    # logging.warning("Empty result")
                    pass
                else:
                    pruned_results.append(res_item)
            if len(pruned_results) == 0:
                logging.warning("Results entirely empty")
                result = ["ERROR404"]
        assert all([len(x)>0 for x in pruned_results]), "Some of the results are empty"
        # print("pruned results", pruned_results)
        if self.debug:
            # if success==False:
            print("Prompt: ", prompt)
            print("Success: ", success)
            print("Response: ", response.content)
            print("Result: ", result)
            print("Pruned results: ", pruned_results)
            print("-------------")
        return list(set(pruned_results))

In [3]:
from prompt_template import PromptBuilder

pf = "/home/lisa/repos/semeval/assets/prompts/tib-core/alltypes_de_abstitle_8_0.txt"
instr = "Extrahiere Schlagwörter aus dem Text."
pb = PromptBuilder(pf, "meta-llama/Meta-Llama-3.1-8B-Instruct", instr)

prompt = pb.build_prompt_by_template("/home/lisa/repos/semeval/assets/prompts/templates/llama.json", debug=True)

/home/lisa/repos/semeval/assets/prompts/tib-core/alltypes_de_abstitle_8_0.txt
Instruction: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

{custom_instruction}
<|eot_id|><|start_header_id|>user<|end_header_id|>


Example_pattern: {text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Keywords pattern: {keywords}<|eot_id|><|start_header_id|>user<|end_header_id|>


Test item pattern: {text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Prompt after instruction:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Extrahiere Schlagwörter aus dem Text.
<|eot_id|><|start_header_id|>user<|end_header_id|>


Prompt after example:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Extrahiere Schlagwörter aus dem Text.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Die Metallurgie des Schweißens : Eisenwerkstoffe - nichteisenmetallische Werkstoffe In der vorliegenden dritten Auflage wurden wesentliche neue Themen praxisgerecht erweitert. I

In [4]:
import pandas as pd

test_data_tibcore = pd.read_csv("/home/lisa/repos/semeval/datasets/tib-core-subjects-Article-Book-Conference-Report-Thesis-de-dev.csv")


In [5]:
i = 15
test_item = test_data_tibcore.iloc[i]
print(test_item)

language                                                          de
split                                                            dev
text_type                                                       Book
subjects                                           tib-core-subjects
idn                                                     3A1623833817
title                   Die Eroberung Mexikos : Cortés und Montezuma
abstract           Dies ist die erste große Darstellung der Erobe...
dcterms:subject    ['gnd:4195062-8', 'gnd:4004069-0', 'gnd:413560...
labels                 ['Eroberung Mexikos', 'Azteken', 'Untergang']
Name: 15, dtype: object


In [6]:
both = test_item["title"] + " " + test_item["abstract"]
title = test_item["title"]
abstract = test_item["title"]

## Modelle

In [7]:
# 	mistralai/Mistral-7B-v0.1
prompt = pb.build_prompt_by_template("/home/lisa/repos/semeval/assets/prompts/templates/chatml.json", debug=True)

Instruction: <|im_start|>system
{custom_instruction}<|im_end|>

Example_pattern: <|im_start|>user
{text}<|im_end|>

Keywords pattern: <|im_start|>assistant
{keywords}<|im_end|>

Test item pattern: <|im_start|>user
{text}<|im_end|>
<|im_start|>assistant

Prompt after instruction:  <|im_start|>system
Extrahiere Schlagwörter aus dem Text.<|im_end|>

Prompt after example:  <|im_start|>system
Extrahiere Schlagwörter aus dem Text.<|im_end|>
<|im_start|>user
Die Metallurgie des Schweißens : Eisenwerkstoffe - nichteisenmetallische Werkstoffe In der vorliegenden dritten Auflage wurden wesentliche neue Themen praxisgerecht erweitert. In erster Linie handelt es sich um zusätzliche Hinweise zu den Phasenumwandlungen (Martensitbildung und Umwandlungen im flüssigen und festen Zustand). Die Ausführungen zu Korrosionsschäden an geschweißten Konstruktionen (vor allem bei denen, die aus "korrosionsbeständigen" Werkstoffen bestehen) erweitert, die erheblich zugenommen haben.Der Umgang mit den häufig abst

In [8]:
comp = Completion({"max_new_tokens": 64, "temperature": 0}, "mistralai/Mistral-7B-v0.1")
comp.completion(prompt.format(text=both))

['Cortés', 'Azteken', 'Mexiko', 'Montezuma']

In [9]:
# teknium/OpenHermes-2.5-Mistral-7B
comp = Completion({"max_new_tokens": 64, "temperature": 0}, "teknium/OpenHermes-2.5-Mistral-7B")
comp.completion(prompt.format(text=both))

['Cortés', 'Aztekenreich', 'Mexiko', 'Eroberung', 'Montezuma']

## Temperature

In [7]:
comp = Completion({"max_new_tokens": 64, "temperature": 0}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Montezuma', 'Eroberung', 'Cortés', 'Mexiko', 'Azteken']

In [8]:
comp = Completion({"max_new_tokens": 128, "temperature": 0.1}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Montezuma', 'Eroberung', 'Cortés', 'Mexiko', 'Azteken']

In [9]:
comp = Completion({"max_new_tokens": 128, "temperature": 0.2}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Eroberung Mexikos', 'Hernán Cortés', 'Moctezuma II.']

In [10]:
comp = Completion({"max_new_tokens": 128, "temperature": 0.3}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Montezuma', 'Eroberung', 'Cortés', 'Mexiko', 'Azteken']

In [11]:
comp = Completion({"max_new_tokens": 128, "temperature": 0.3}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Eroberung Mexikos', 'Hernán Cortés', 'Moctezuma II.']

In [12]:
comp = Completion({"max_new_tokens": 128, "temperature": 0.4}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Eroberung', 'Conquistador', 'Mexiko', 'Geschichte', 'Azteken']

In [13]:
comp = Completion({"max_new_tokens": 128, "temperature": 0.5}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Eroberung Mexikos', 'Hernán Cortés', 'Moctezuma II.']

In [14]:
comp = Completion({"max_new_tokens": 128, "temperature": 0.7}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Montezuma', 'Cortés']

In [15]:
comp = Completion({"max_new_tokens": 128, "temperature": 0.9}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Eroberung von Mexiko durch die Spanier']

In [16]:
comp = Completion({"max_new_tokens": 128, "temperature": 0.9}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

['Montezuma', 'Mexiko', 'Eroberung', 'Cortés']

## Instruction

In [19]:
instr = "Extrahiere Schlagwörter aus dem Text."
pb = PromptBuilder(pf, "meta-llama/Meta-Llama-3.1-8B-Instruct", instr)
prompt = pb.build_prompt_by_template("/home/lisa/repos/semeval/assets/prompts/templates/llama.json", debug=False)
comp = Completion({"max_new_tokens": 64, "temperature": 0}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

/home/lisa/repos/semeval/assets/prompts/tib-core/alltypes_de_abstitle_8_0.txt


['Montezuma', 'Eroberung', 'Cortés', 'Mexiko', 'Azteken']

In [18]:
instr = "Du bist ein Experte, der in einer Bibliothek arbeitet. Du bist für die Inhaltserschließung zuständig. Extrahiere Schlagwörter aus dem Text."
pb = PromptBuilder(pf, "meta-llama/Meta-Llama-3.1-8B-Instruct", instr)
prompt = pb.build_prompt_by_template("/home/lisa/repos/semeval/assets/prompts/templates/llama.json", debug=False)
comp = Completion({"max_new_tokens": 64, "temperature": 0}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

/home/lisa/repos/semeval/assets/prompts/tib-core/alltypes_de_abstitle_8_0.txt


['Montezuma', 'Eroberung', 'Cortés', 'Mexiko', 'Azteken']

In [20]:
instr = "Du bist ein Experte, der in einer Bibliothek arbeitet. Du bist für die Inhaltserschließung zuständig. Extrahiere die wichtigsten Themen des Textes. Verwende sowohl spezifische Schlagwörter als auch abstrakte Konzepte."
pb = PromptBuilder(pf, "meta-llama/Meta-Llama-3.1-8B-Instruct", instr)
prompt = pb.build_prompt_by_template("/home/lisa/repos/semeval/assets/prompts/templates/llama.json", debug=False)
comp = Completion({"max_new_tokens": 64, "temperature": 0}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

/home/lisa/repos/semeval/assets/prompts/tib-core/alltypes_de_abstitle_8_0.txt


['Montezuma', 'Eroberung', 'Cortés', 'Mexiko', 'Azteken']

In [21]:
instr = "Du bist ein Experte, der in einer Bibliothek arbeitet. Du bist für die Inhaltserschließung zuständig. Extrahiere die wichtigsten Personen, Orte, Ereignisse und Themen aus dem Text."
pb = PromptBuilder(pf, "meta-llama/Meta-Llama-3.1-8B-Instruct", instr)
prompt = pb.build_prompt_by_template("/home/lisa/repos/semeval/assets/prompts/templates/llama.json", debug=False)
comp = Completion({"max_new_tokens": 64, "temperature": 0}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

/home/lisa/repos/semeval/assets/prompts/tib-core/alltypes_de_abstitle_8_0.txt


['Eroberung Mexikos', 'Hernán Cortés', 'Moctezuma II.']

## Best_of

In [37]:
comp = Completion_Multiple({"max_new_tokens": 64, "best_of": 2, "temperature":0.1}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

AT ['Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Cortés, Montezuma']
AT ['Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung Mexikos, Hernán Cortés, Moctezuma II.']
AT ['Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Cortés, Montezuma']
AT ['Eroberung, Mexiko, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung Mexikos, Hernán Cortés, Moctezuma II.', 'Eroberung Mexikos, Hernán Cortés, Moctezuma II.']
AT ['Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung, Mexiko, Cortés, Montezuma', 'Eroberung Mexikos, Hernán Cortés, Moctezuma II.']
AT ['Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
Error!


[]

In [39]:
comp = Completion_Multiple({"max_new_tokens": 64, "best_of": 3, "temperature":0.1}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

AT ['Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung Mexikos, Hernán Cortés, Moctezuma II.', 'Eroberung, Mexiko, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung, Mexiko, Cortés, Montezuma', 'Eroberung, Mexiko, Cortés, Montezuma', 'Eroberung, Mexiko, Cortés, Montezuma']
AT ['Eroberung, Mexiko, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung Mexikos, Hernán Cortés, Moctezuma II.', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung, Mexiko, Cortés, Montezuma', 'Eroberung Mexikos, Hernán Cortés, Montezuma', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung Mexikos, Hernán Cortés, Moctezuma II.', 'Eroberung Mexikos, Hernán Cortés, Moctezuma II.', 'Eroberung, Mexiko, Azteken, Cortés, Mon

[]

In [40]:
comp = Completion_Multiple({"max_new_tokens": 64, "best_of": 3, "temperature":0.5}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

AT ['Eroberung Mexikos, Hernán Cortés, Moctezuma II.', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma', 'Eroberung von Mexiko, Hernán Cortés, Moctezuma II.']
AT ['Cortés, Montezuma, Eroberung Mexikos', 'Cortés, Montezuma, Azteken, Eroberung Mexikos', 'Azteken, Mexiko, Eroberung, Hernán Cortés, Moctezuma']
AT ['Eroberung Mexikos, Hernán Cortés, Moctezuma II.', 'Eroberung, Azteken, Cortés, Montezuma', 'Eroberung Mexikos, Hernán Cortés, Moctezuma II.']
AT ['Eroberung Mexikos, Azteken, Cortés, Montezuma', 'Eroberung, Mexiko, Cortés, Montezuma', 'Eroberung, Mexiko, Azteken']
AT ['Eroberung, Mexiko, Cortés, Montezuma, Azteken', 'Eroberung Mexikos, Hernán Cortés, Montezuma', 'Eroberung, Mexiko, Hernán Cortés, Montezuma']
AT ['Eroberung, Mexiko, Cortés, Montezuma', 'Eroberung Mexikos, Hernán Cortés, Moctezuma II., Azteken', 'Eroberung, Mexiko, Azteken, Cortés, Montezuma']
AT ['Eroberung Mexikos, Hernán Cortés, Moctezuma II.', 'Eroberung Mexikos, Cortes, Montezuma', 'Azteken, Mexiko, Eroberung,

[]

In [41]:
comp = Completion_Multiple({"max_new_tokens": 64, "best_of": 3, "temperature":0.8}, "meta-llama/Meta-Llama-3.1-8B-Instruct")
comp.completion(prompt.format(text=both))

AT ['Aztekenreich, Eroberung Mexikos', 'Mexiko, Konquistador, Spanische Eroberung, Azteken', 'Eroberung, Mexiko, Montezuma, Cortés']
AT ['Montezuma, Cortés', 'Eroberung, Mexiko, Mexikanische Empfehlungen', 'Mexiko, Eroberung, Cortés, Montezuma']
AT ['Cortes, Montezuma, Eroberung Mexikos', 'Eroberung, Montezuma, Cortés, Azteken', 'Eroberung, Mexiko, Aztekenreiche, Conquistador, Hernan Cortes']
AT ['Mexiko, Azteken, Cortés, Montezuma', 'Mexico, Azteken, Eroberung, Cortés, Montezuma', 'Cortés, Montezuma']
AT ['Cortés, Montezuma, Azteken', 'Eroberung Mexikos, Hernán Cortés, Montezuma', 'Eroberung Mexikos, Azteken, Hernán Cortés, Moctezuma']
AT ['Cortés, Montezuma, Azteken, Eroberung Mexikos', 'Aztekenreich, Mexiko, Conquistadore, Heerscharen, Spanier', 'Kolumbus']
AT ['Montezuma, Eroberung, Mexiko, Cortés', 'Neuere Geschichte, Mexiko, Mexikogeschichte, Kolumbus', 'Mexiko, Eroberung, Azteken, Cortés, Montezuma']
AT ['Mexiko, Eroberung, Cortés, Montezuma, Azteken', 'Montezuma, Cortés', 'Erob

[]

# vLLM Model testing

In [9]:
from vllm import LLM, SamplingParams
from vllm.engine.arg_utils import EngineArgs
from dataclasses import asdict

vllm_samplingparams = SamplingParams(
            max_tokens = 128,
            temperature= 0,
            presence_penalty= 0,
            frequency_penalty= 0,
            repetition_penalty= 1,
            top_p= 1,
            best_of= 1,
        )

vllm_engineargs = EngineArgs(
            model="openGPT-X/Teuken-7B-instruct-commercial-v0.4", 
            download_dir=".cache/huggingface/hub",
            task="generate",
            gpu_memory_utilization= 0.7, 
            tensor_parallel_size=2,
            dtype="float16",
            trust_remote_code=True
        )

In [10]:
llm = LLM(**asdict(vllm_engineargs))
        

WARNING 11-29 14:49:58 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 11-29 14:49:58 config.py:1020] Defaulting to use mp for distributed inference
INFO 11-29 14:49:58 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='openGPT-X/Teuken-7B-instruct-commercial-v0.4', speculative_config=None, tokenizer='openGPT-X/Teuken-7B-instruct-commercial-v0.4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir='.cache/huggingface/hub', load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=F

KeyboardInterrupt: 

In [11]:
llm.generate(prompt)

NameError: name 'llm' is not defined

[E1129 14:59:57.607575582 socket.cpp:1011] [c10d] The client socket has timed out after 600000ms while trying to connect to (127.0.0.1, 36253).
[W1129 14:59:57.608343123 TCPStore.cpp:358] [c10d] TCP client failed to connect/validate to host 127.0.0.1:36253 - retrying (try=0, timeout=600000ms, delay=31479ms): The client socket has timed out after 600000ms while trying to connect to (127.0.0.1, 36253).
Exception raised from throwTimeoutError at ../torch/csrc/distributed/c10d/socket.cpp:1013 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x96 (0x7f6a12cb9446 in /home/lisa/.conda/envs/llms/lib/python3.12/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x15e04c6 (0x7f69fdf464c6 in /home/lisa/.conda/envs/llms/lib/python3.12/site-packages/torch/lib/libtorch_cpu.so)
frame #2: <unknown function> + 0x6029d95 (0x7f6a0298fd95 in /home/lisa/.conda/envs/llms/lib/python3.12/site-packages/torch/lib/libtorch_cpu.so)
frame #3: <unknown functio

(VllmWorkerProcess pid=2967603) ERROR 11-29 15:09:38 multiproc_worker_utils.py:229] Exception in worker VllmWorkerProcess while processing method init_device.
(VllmWorkerProcess pid=2967603) ERROR 11-29 15:09:38 multiproc_worker_utils.py:229] Traceback (most recent call last):
(VllmWorkerProcess pid=2967603) ERROR 11-29 15:09:38 multiproc_worker_utils.py:229]   File "/home/lisa/.conda/envs/llms/lib/python3.12/site-packages/vllm/executor/multiproc_worker_utils.py", line 223, in _run_worker_process
(VllmWorkerProcess pid=2967603) ERROR 11-29 15:09:38 multiproc_worker_utils.py:229]     output = executor(*args, **kwargs)
(VllmWorkerProcess pid=2967603) ERROR 11-29 15:09:38 multiproc_worker_utils.py:229]              ^^^^^^^^^^^^^^^^^^^^^^^^^
(VllmWorkerProcess pid=2967603) ERROR 11-29 15:09:38 multiproc_worker_utils.py:229]   File "/home/lisa/.conda/envs/llms/lib/python3.12/site-packages/vllm/worker/worker.py", line 145, in init_device
(VllmWorkerProcess pid=2967603) ERROR 11-29 15:09:38 m

In [2]:
user="Hi!"
lang_code = "DE"
system_messages={
            "EN": "A chat between a human and an artificial intelligence assistant."
            " The assistant gives helpful and polite answers to the human's questions.",
            "DE": "Ein Gespräch zwischen einem Menschen und einem Assistenten mit künstlicher Intelligenz."
            " Der Assistent gibt hilfreiche und höfliche Antworten auf die Fragen des Menschen.",
        }
 
prompt = f"System: {system_messages[lang_code]}\nUser: {user}\nAssistant:"
prompt

'System: Ein Gespräch zwischen einem Menschen und einem Assistenten mit künstlicher Intelligenz. Der Assistent gibt hilfreiche und höfliche Antworten auf die Fragen des Menschen.\nUser: Hi!\nAssistant:'

# JSON


In [8]:
import json

data = {
    "dcterms_subject": 
    ["gnd1",
    "gnd2", 
    "gnd3"]}
with open("../assets/test.json", "w") as f:
    json.dump(data, f, indent=4)
    

In [6]:
import json

data = {
    "dcterms_subject": [
        "gnd:4068097-6",
        "gnd:4033447-8",
        "gnd:4123065-6",
        "gnd:4045791-6",
        "gnd:4126976-7",
        "gnd:5689012-3",
        "gnd:9764123-1",
        "gnd:2347812-9",
        "gnd:8923476-2",
        "gnd:1347892-4",
        "gnd:7623914-5",
        "gnd:8741236-8",
        "gnd:2314976-6",
        "gnd:5698347-7",
        "gnd:7812396-1",
        "gnd:4578123-5",
        "gnd:9812364-9",
        "gnd:6421398-2",
        "gnd:4738126-7",
        "gnd:3128796-3",
        "gnd:3879124-1",
        "gnd:4318976-5",
        "gnd:1276389-8",
        "gnd:5362789-4"
    ]
}

with open('../assets/output.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [15]:
from transformers import AutoTokenizer
import pandas as pd

tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
ds = pd.read_csv("/home/lisa/repos/backup_repos/main_semeval/datasets/all-subjects-tib-core-subjects-Article-Book-Conference-Report-Thesis-en-de-dev_sample1000.csv")
ds["both"] = ds["title"] + " " + ds["abstract"]
texts = ds["both"].tolist()
lengths_llama = [len(tokenizer_llama.tokenize(text)) for text in texts]
del tokenizer_llama
tokenizer_mistral = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
lengths_mistral = [len(tokenizer_mistral.tokenize(text)) for text in texts]
with open("/home/lisa/repos/backup_repos/main_semeval/assets/prompts/english/english_2_12.txt") as f:
    pr = f.read()
    print(len(tokenizer_mistral.tokenize(pr)))
del tokenizer_mistral

lengths_df = pd.DataFrame({"llama": lengths_llama, "mistral": lengths_mistral})
lengths_df.describe()

4016


,llama,mistral
count,1008.000000,1008.000000
mean,286.324405,320.572421
std,196.778506,219.647010
min,21.000000,25.000000
25%,151.750000,170.000000
50%,250.500000,280.000000
75%,375.500000,422.000000
max,2622.000000,2889.000000


In [ ]:
from transformers import AutoTokenizer
import pandas as pd

tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
ds = pd.read_csv("/home/lisa/repos/backup_repos/main_semeval/datasets/all-subjects-tib-core-subjects-Article-Book-Conference-Report-Thesis-en-de-dev_sample1000.csv")
ds["both"] = ds["title"] + " " + ds["abstract"]
texts = ds["both"].tolist()
lengths_llama = [len(tokenizer_llama.tokenize(text)) for text in texts]
del tokenizer_llama
# tokenizer_mistral = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
# lengths_mistral = [len(tokenizer_mistral.tokenize(text)) for text in texts]
# with open("/home/lisa/repos/backup_repos/main_semeval/assets/prompts/english/english_2_12.txt") as f:
#     pr = f.read()
#     print(len(tokenizer_mistral.tokenize(pr)))
# del tokenizer_mistral

lengths_df = pd.DataFrame({"llama": lengths_llama})
lengths_df.describe()

In [1]:
import os
import json
import pandas as pd

# Initialize an empty list to store the data
data = []

# Traverse the directory structure
for root, dirs, files in os.walk('../results/'):
    for file in files:
        if file == 'mapping_stats.json':
            # Extract model and prompt from the directory structure
            parts = root.split(os.sep)
            if len(parts) >= 2:
                model = parts[-2]
                prompt = parts[-1]
                
                # Read the JSON file
                with open(os.path.join(root, file), 'r') as f:
                    stats = json.load(f)
                
                # Append the data to the list
                data.append({
                    'model': model,
                    'prompt': prompt,
                    'n_keywords': stats.get('n_keywords', 0),
                    'n_perfect_matches': stats.get('n_perfect_matches', 0),
                    'n_only_vector_matches': stats.get('n_only_vector_matches', 0),
                    'n_not_in_allowed_subjects': stats.get('n_not_in_allowed_subjects', 0)
                })

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a file
df.to_csv('../assets/combined_stats.csv', index=False)